In [ ]:
#!pip install snntorch
#!pip install opencv-python
#!pip install brevitas
#!pip install onnx==1.8.1  
!pip install onnxoptimizer

In [ ]:
###### RUN
# Create dataset using HDF5 Format
# Errors due to dataset being not discrete. Hand pick ictal samples.
# iNVERSE- SPIKES IN  BLACK, nO inverse - SPIKES IN WHITE
#!pip install snntorch
# imports
import snntorch as snn
from snntorch import surrogate
from snntorch import backprop
from snntorch import functional as SF
from snntorch import utils
from snntorch import spikeplot as splt

import onnx   #1.8.1   #Import ONNX before torch
import onnxoptimizer as opt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F

import matplotlib.pyplot as plt
import numpy as np
import itertools
import brevitas.nn as qnn
import torch.onnx
from brevitas.export import FINNManager

In [ ]:
###### RUN
import glob
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader,ConcatDataset
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor

pat_id = 'pat_442'

class CustomDataset(Dataset):
    def __init__(self, transform=None):
        #self.imgs_path = "./TF_images_ictal_nonictal/"
        #self.imgs_path = "./TF_plot_images/pat_139/"
        #self.imgs_path = "./TF_plot_images/pat_253/"
        self.imgs_path = './TF_plot_images_rand/' + pat_id + '/train_data/' 
        file_list = glob.glob(self.imgs_path + "*")
        print(file_list)
        self.data = []
        for class_path in file_list:
            class_name = class_path.split("/")[-1]
            for img_path in glob.glob(class_path + "/*.png"):
                self.data.append([img_path, class_name])
        print('self.data length',len(self.data))
        self.class_map = {"non_ictal" : 0, "ictal": 1}
        self.img_dim = (28, 28)
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_path, class_name = self.data[idx]
        img = cv2.imread(img_path)                      # Load Image 
        #print('shape for original image',img.shape)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)    # in grayscale
        #img = img[:,:,2]                                # take red component only
        #print('shape for image grayscale',img.shape)
        img = img[60:60+366, 81:81+396]                 # Crop image  
        #print('shape for image cropped',img.shape)
        img = cv2.bitwise_not(img)                      # Negativ-positiv inversion #TODO: improve this
        #print('shape for image bitwise not',img.shape)
        img = cv2.resize(img, self.img_dim)             # resize
        #print('shape for image resize',img.shape)
        #img_tensor = torch.from_numpy(img)
        #img_tensor = torch.unsqueeze(img_tensor, 0).float()
        #print('shape for image before',img_tensor.shape)
        if self.transform is not None:
            img_tensor = self.transform(img)
            img_tensor = img_tensor.float()
            #print('return type of transform',img_tensor.shape, img_tensor.dtype)
        class_id = self.class_map[class_name]
        #if class_id == 1:
        #    cv2.imshow('rgb image', img)
        #    cv2.waitKey(5)
        #print('Label is ', class_id)
        #img_tensor = img_tensor.permute(2, 0, 1)     # for BGR image
        class_id = torch.tensor([class_id])
        #print('data', img_tensor.float().shape , 'targets', class_id.float().shape)
        return img_tensor.float(), class_id.float()

In [ ]:
###### RUN
# imports
import snntorch as snn
from snntorch import spikeplot as splt
from snntorch import spikegen
from IPython.display import HTML

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
import numpy as np
import itertools

# dataloader arguments
batch_size = 64
#dtype = torch.float
#device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device = 'cpu'

# Define a transform
transform = transforms.Compose([
            #transforms.Resize((32, 32)),
            #transforms.Grayscale(),     #Returns PIL image
            transforms.ToTensor(),
            transforms.Normalize((0,), (1,))  # ToTensor and Normalize work together
        ])

dataset = CustomDataset(transform = transform)

In [ ]:
###### RUN
# Making training test validation split and creating dataloader
from sklearn.model_selection import LeavePGroupsOut  #TODO; Do LOO split for validation
from torch.utils.data import DataLoader, WeightedRandomSampler
from sklearn.utils.class_weight import compute_class_weight
import pandas as pd

def train_test_split(dataset):
    """Split dataset into train and test 
    
    Parameters
    ----------
    dataset : ConcatDataset
        The dataset to split.
        
    Returns
    -------
    ConcatDataset
        The training data and the testing data.
    """
    print('Total images', len(dataset))
    train_size = int(0.99 * len(dataset))
    test_size = len(dataset) - train_size
    print('Train and test size', train_size, test_size)
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
    return train_dataset, test_dataset

torch.manual_seed(87)
np.random.seed(87)
train_ds, test_ds = train_test_split(dataset)
print('Number of examples in each set:')
print(f'Training: {len(train_ds)}')
print(f'Test: {len(test_ds)}')
print(train_ds)

#Class weights
train_y = pd.Series([y for _,y in train_ds])
#print('train_y train 0/1: {}/{}'.format((train_y == 0).sum(), (train_y == 1).sum()))

class_sample_count = torch.tensor(
    [(train_y == t).sum() for t in torch.unique(torch.tensor(train_y), sorted=True)])
print('train class 0/1 weightage', class_sample_count)

#WeightedSampler
weights = 1. / class_sample_count.float()
print('weights', weights)
weights[1] = weights[1]/2.0
print('weights', weights)
samples_weights = torch.tensor([weights[t.long()] for t in torch.tensor(train_y)])
print('sample weights', samples_weights)
sampler = WeightedRandomSampler( weights=samples_weights, num_samples=len(train_ds), replacement=True)

# Create dataloaders
num_workers = 0  
train_loader = DataLoader(
    train_ds, batch_size=batch_size,  num_workers=num_workers, sampler=sampler) # Todo: shuffle?
#loader_valid = DataLoader(
#    valid_ds, batch_size=valid_batch_size, shuffle=False, num_workers=num_workers)
valid_loader = DataLoader(
    dataset_test, batch_size=batch_size, shuffle=False, num_workers=num_workers)
print(f'Training: {len(train_ds)}')
print(f'Test: {len(dataset_test)}')

In [ ]:
# Iterate DataLoader and check class balance for each batch
for i, (x, y) in enumerate(train_loader):
    print("batch index {}, 0/1: {}/{}".format(
        i, (y == 0).sum(), (y == 1).sum()))

In [ ]:
from brevitas.nn import QuantIdentity
from brevitas.nn import QuantConv2d
from brevitas.quant import Int8Bias
from brevitas.nn import QuantDropout
from brevitas.nn import QuantReLU
from brevitas.nn import QuantSigmoid
from brevitas.quant.scaled_int import Int8ActPerTensorFloat
%env BREVITAS_JIT=1
%env BREVITAS_IGNORE_MISSING_KEYS=1 

In [ ]:
###### RUN
def reset_weights(m):
    '''Try resetting model weights to avoid weight leakage. '''
    for layer in m.children():
        if hasattr(layer, 'reset_parameters'):
            print(f'Reset trainable parameters of layer = {layer}')
            layer.reset_parameters()

In [ ]:
# CREATING NEURAL NETWORK with parameterization for freq = 256Hz and t= 2sec : based on Farrokh's
from torch import nn
import torch.nn.functional as F

class IEEGSeizureDetection_cnv(nn.Module):
   
    def __init__(self):
        super(IEEGSeizureDetection_cnv, self).__init__()

        weight_bit_width = 4
        act_bit_width = 4
        
        #Layer1
        self.quant_identity = QuantIdentity(return_quant_tensor=True)
        self.conv1 = qnn.QuantConv2d(in_channels=1, out_channels= 16, kernel_size= 5,
                                     bias=True,
                                     #weight_bit_width=weight_bit_width,
                                     #input_quant=Int8ActPerTensorFloat, 
                                     #weight_quant=Int8ActPerTensorFloat,
                                     bias_quant=Int8Bias,
                                     #output_quant=Int8ActPerTensorFloat,
                                     return_quant_tensor=True)
        #print(self.conv1)
        self.batchnorm1 = nn.BatchNorm2d(16)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        #self.dropout = QuantDropout(0.2)
        self.relu1 = QuantReLU(return_quant_tensor=True)
        
        #Layer2
        self.conv2 = qnn.QuantConv2d(in_channels=16, out_channels=64 ,kernel_size= 5,
                                     bias=True,
                                     #weight_bit_width=weight_bit_width,
                                     #input_quant=Int8ActPerTensorFloat, 
                                     bias_quant=Int8Bias,
                                     #output_quant=Int8ActPerTensorFloat,
                                     return_quant_tensor=True)
        #print(self.conv2)
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        self.relu2 = QuantReLU(return_quant_tensor=True)
        
        #Layer3
        self.conv3 = qnn.QuantConv2d(in_channels=64, out_channels=64 , kernel_size=1,
                                     bias=True,
                                     #weight_bit_width=weight_bit_width,
                                     #input_quant=Int8ActPerTensorFloat, 
                                     bias_quant=Int8Bias,
                                     #output_quant=Int8ActPerTensorFloat,
                                     return_quant_tensor=True)
        #print(self.conv3)
        self.batchnorm3 = nn.BatchNorm2d(64)
        #self.maxpool3 = nn.MaxPool2d(kernel_size=2)
        self.relu3 = QuantReLU(return_quant_tensor=True)
        '''
        #Layer4
        self.conv4 = qnn.QuantConv2d(in_channels=10, out_channels=10 , kernel_size=(1,5),
                                     bias=True,
                                     #weight_bit_width=weight_bit_width,
                                     #input_quant=Int8ActPerTensorFloat, 
                                     bias_quant=Int8Bias,
                                     #output_quant=Int8ActPerTensorFloat,
                                     return_quant_tensor=True)
        #print(self.conv4)
        self.batchnorm4 = nn.BatchNorm2d(10)
        self.relu4 = QuantReLU(return_quant_tensor=True)
        '''
        #Layer5
        self.fc1 = qnn.QuantLinear(in_features=64*4*4, out_features= 128, 
                                   bias=True,   
                                   #weight_bit_width=weight_bit_width,
                                   #output_quant=Int8ActPerTensorFloat,
                                   bias_quant=Int8Bias,
                                   return_quant_tensor=True)
        #print(self.fc1)
        self.relu5 = QuantReLU(return_quant_tensor=True)
        
        #Layer6
        self.fc2 = qnn.QuantLinear(in_features=128, out_features=1, 
                                   bias=True,  
                                   #weight_bit_width=weight_bit_width,
                                   #output_quant=Int8ActPerTensorFloat,
                                   bias_quant=Int8Bias,
                                   return_quant_tensor=True)
        #print(self.fc2)
        #self.relu5 = QuantReLU(return_quant_tensor=True)
        #self.relu6 = QuantSigmoid(return_quant_tensor=True)
        '''
        #Layer7
        self.fc3 = qnn.QuantLinear(in_features=5, 
                                   #out_features=1, 
                                   #input_quant=Int8ActPerTensorFloat, 
                                   out_features=2,
                                   bias=True, 
                                   #weight_bit_width=weight_bit_width,
                                   #output_quant=Int8ActPerTensorFloat,
                                   bias_quant=Int8Bias,
                                   return_quant_tensor=True)'''
        #print(self.fc3)
        #self.sigmoid = QuantSigmoid(return_quant_tensor=True)
        #self.sigmoid = nn.Sigmoid()
        #'''
        # Non Quantized

    def forward(self, x):
        """Forward pass.
        
        Parameters
        ---------
        x: torch.Tensor
            Batch of EEG windows of shape (batch_size, n_channels, n_times).
        """
        
        x = self.quant_identity(x)  # UNCOMMENT FOR QUANTIZATION 
        #print(f"Input has QuantTensor:\n {x.is_valid} \n")
        x = self.conv1(x)
        #print(f"CONV1:\n {x} \n")
        #print(f"CONV1 has QuantTensor:\n {x.is_valid} \n")
        x = self.batchnorm1(x)
        #print(f"BN1 has QuantTensor:\n {x.is_valid} \n")
        x = self.relu1(x)
        #print(f"R1 has QuantTensor:\n {x.is_valid} \n")
        x = self.maxpool1(x)
        #print(f"MP1 has QuantTensor:\n {x.is_valid} \n")
        #x = self.dropout(x)
        #print(f"DR1 has QuantTensor:\n {x.is_valid} \n")
        x = self.conv2(x)
        #print(f"CONV2:\n {x} \n")
        #print(f"CONV2 has QuantTensor:\n {x.is_valid} \n")
        x = self.batchnorm2(x)
        #print(f"BN2 has QuantTensor:\n {x.is_valid} \n")
        x = self.relu2(x)
        #print(f"R2 has QuantTensor:\n {x.is_valid} \n")
        x = self.maxpool2(x)
        x = self.conv3(x)
        #print(f"CONV2:\n {x} \n")
        #print(f"CONV2 has QuantTensor:\n {x.is_valid} \n")
        x = self.batchnorm3(x)
        #print(f"BN2 has QuantTensor:\n {x.is_valid} \n")
        x = self.relu3(x)
        #print(f"MP2 has QuantTensor:\n {x.is_valid} \n")
        #x = self.dropout(x)
        #print(f"DR2 has QuantTensor:\n {x.is_valid} \n")
        x = self.relu5(self.fc1(x.flatten(start_dim=1)))
        #print(f"FC1:\n {x} \n")
        #print(f"FC1 has QuantTensor:\n {x.is_valid} \n")
        #x = self.relu6(self.fc2(x))
        x = self.fc2(x)
        #print(f"FC2:\n {x} \n")
        #print(f"FC2 has QuantTensor:\n {x.is_valid} \n")
        #x = self.sigmoid(self.fc3(x))
        #x = self.fc3(x)
        #print(f"FC3:\n {x} \n")
        #print(f"FC3 has QuantTensor:\n {x.is_valid} \n")

        return x

In [ ]:
class IEEGSeizureDetection(nn.Module):
    def __init__(self):
        super().__init__()
        input_size = 784
        self.quant_inp = qnn.QuantIdentity(bit_width=4, return_quant_tensor=True)
        self.fc1 = qnn.QuantLinear(input_size, 512, weight_bit_width=3, bias=True,bias_quant=Int8Bias,return_quant_tensor=True)
        self.relu1 = qnn.QuantReLU(bit_width=4, return_quant_tensor=True)
        self.fc2 = qnn.QuantLinear(512, 256, weight_bit_width=3, bias=True,bias_quant=Int8Bias,return_quant_tensor=True)
        self.relu2 = qnn.QuantReLU(bit_width=4, return_quant_tensor=True)
        self.fc3 = qnn.QuantLinear(256, 128, weight_bit_width=3, bias=True,bias_quant=Int8Bias,return_quant_tensor=True)
        self.relu3 = qnn.QuantReLU(bit_width=4, return_quant_tensor=True)
        self.fc4 = qnn.QuantLinear(128, 64, weight_bit_width=3, bias=True,bias_quant=Int8Bias,return_quant_tensor=True)
        self.relu4 = qnn.QuantReLU(bit_width=4, return_quant_tensor=True)
        self.fc5 = qnn.QuantLinear(64,1, weight_bit_width=3, bias=False)
        self.dropout = nn.Dropout(p=0.2)
        #return_quant_tensor = True to propogate QuantTensor across
        #Quant_relu is stateful, so instantiate all of them separately
        
    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.quant_inp(x)
        x = self.dropout(self.relu1(self.fc1(x)))
        x = self.dropout(self.relu2(self.fc2(x)))
        x = self.dropout(self.relu3(self.fc3(x)))
        x = self.dropout(self.relu4(self.fc4(x)))
        x = self.fc5(x)
        #x = nn.Sigmoid(x)
        return x

In [ ]:
###### RUN
# CSNN
# neuron and simulation parameters
spike_grad = surrogate.fast_sigmoid(slope=25)
beta = 0.8
num_steps = 50
print(batch_size)
'''
# Define Network
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        # Initialize layers
        self.conv1 = nn.Conv2d(1, 12, 5)
        self.lif1 = snn.Leaky(beta=beta, spike_grad=spike_grad)
        self.conv2 = nn.Conv2d(12, 64, 5)
        self.lif2 = snn.Leaky(beta=beta, spike_grad=spike_grad)
        self.fc1 = nn.Linear(64*4*4, 2)
        self.lif3 = snn.Leaky(beta=beta, spike_grad=spike_grad)

    def forward(self, x):

        # Initialize hidden states and outputs at t=0
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky() 
        mem3 = self.lif3.init_leaky()

        # Record the final layer
        spk3_rec = []
        mem3_rec = []

        for step in range(num_steps):
            cur1 = F.max_pool2d(self.conv1(x), 2)
            spk1, mem1 = self.lif1(cur1, mem1)
            cur2 = F.max_pool2d(self.conv2(spk1), 2)
            spk2, mem2 = self.lif2(cur2, mem2)
            cur3 = self.fc1(spk2.view(batch_size, -1))
            spk3, mem3 = self.lif3(cur3, mem3)

            spk3_rec.append(spk3)
            mem3_rec.append(mem3)

        return torch.stack(spk3_rec), torch.stack(mem3_rec)

#  Initialize Network 
net = Net().to(device)
'''
num_bits = 4
net = nn.Sequential(
                    qnn.QuantConv2d(1, 16, 5, bias=False, return_quant_tensor=True), #try 16
                    #nn.Conv2d(1, 16, 5),
                    nn.BatchNorm2d(16),
                    nn.MaxPool2d(2),
                    #nn.ReLU(),
                    qnn.QuantReLU(return_quant_tensor=True),
                    #snn.Leaky(beta=beta, threshold=1.0, spike_grad=spike_grad, init_hidden=True),
                    qnn.QuantConv2d(16, 64, 5, bias=False,  return_quant_tensor=True),
                    #nn.Conv2d(16, 64, 5),
                    nn.BatchNorm2d(64),
                    nn.MaxPool2d(2),
                    #nn.ReLU(),
                    qnn.QuantReLU(return_quant_tensor=True),
                    #snn.Leaky(beta=beta, threshold=1.0, spike_grad=spike_grad, init_hidden=True),
                    nn.Flatten(),
                    qnn.QuantLinear(64 * 4 * 4, 128, bias=False, return_quant_tensor=True),
                    #nn.Linear(64*4*4, 128),
                    #nn.ReLU(),
                    qnn.QuantReLU(return_quant_tensor=True),
                    #snn.Leaky(beta=beta, threshold=1.0, spike_grad=spike_grad, init_hidden=True),
                    qnn.QuantLinear(128, 1, bias=False),
                    #nn.Linear(128, 1),
                    nn.Sigmoid(),
                    #qnn.QuantSigmoid(return_quant_tensor=True)
                    #snn.Leaky(beta=beta, threshold=1.0, spike_grad=spike_grad, init_hidden=True, output=True)
                    ).to(device)
net.apply(reset_weights)

In [ ]:
model = IEEGSeizureDetection_cnv()
print(f'Using device \'{device}\'.')
model = model.to(device)
model.apply(reset_weights)

In [ ]:
print(device)


In [ ]:
from sklearn.metrics import accuracy_score
def get_accuracy(y_true, y_prob):
    print('True',y_true,'Pred',y_prob)
    assert(len(y_true) == len(y_prob))
    threshold = [0.5] * len(y_true)
    accuracy = accuracy_score(y_true, y_prob > threshold)
    return accuracy
def get_accuracy_(y_true, y_prob):
    #assert y_true.ndim == 1 and y_true.size() == y_prob.size()
    y_prob = [t > 0.5 for t in y_prob]
    print('True',y_true,'Pred',y_prob, len(y_true), len(y_prob))
    accuracy = accuracy_score(y_true, y_prob)
    return accuracy
    #return (y_true == y_prob).sum().item() / y_true.size(0)

optimizer = torch.optim.Adam(model.parameters(), lr= 3.0e-3, betas=(0.9, 0.999))
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 
                                                        T_max=4690, 
                                                        eta_min=0, 
                                                       last_epoch=-1)
#criterion = nn.CrossEntropyLoss()
criterion = nn.BCELoss()
num_epochs = 100
m = nn.Sigmoid()

# keeping-track-of-losses 
train_losses = []
valid_losses = []
y_prob_all, y_true_all, y_pred_all = list(), list(), list()
y_prob_all_test, y_true_all_test, y_pred_all_test = list(), list(), list()
for epoch in range(1, num_epochs + 1):
    
    # keep-track-of-training-and-validation-loss
    train_loss = 0.0
    valid_loss = 0.0
    correct_train = 0
    total_train = 0
    correct_test = 0
    total_test = 0
    # training-the-model
    model.train()

    for data, target in train_loader:
        # move-tensors-to-GPU 
        data = data.to(device)
        #print('data', data.shape)
        target = target.to(device)
        #print('target', target.shape)
        
        # clear-the-gradients-of-all-optimized-variables
        optimizer.zero_grad()

        # forward-pass: compute-predicted-outputs-by-passing-inputs-to-the-model
        output = model(data)
        output_s = m(output)
        #print('output', output.shape)

        #print(pred)
        # calculate-the-batch-loss
        loss = criterion(output_s, target)
        loss.backward()
        optimizer.step()
        
        predicted = output_s > 0.5
        #print('Predicted', predicted, predicted.shape)
        total_train += target.size(0)
        correct_train += (predicted == target).sum().item()
        y_prob_all.append(torch.squeeze(output_s).detach().cpu().numpy())
        y_true_all.append(torch.squeeze(target).cpu().numpy())
        y_pred_all.append(torch.squeeze(predicted).cpu().numpy().astype(float) )
        #y_pred_all.append((out_sigmoid.cpu().numpy() > 0.5))
        # update-training-loss
        train_loss += loss.item() * data.size(0)
    
    y_prob = np.concatenate(y_prob_all)
    y_true = np.concatenate(y_true_all)
    y_pred = np.concatenate(y_pred_all)
    print('what true',y_true.shape, y_true, 'pred', y_pred.shape, y_pred)
    acc_train = accuracy_score(y_true, y_pred)
    #acc = np.mean(y_true == y_pred)  
    perf_train = 100 * correct_train / total_train
    print('Training accuracy', acc_train)
    
    # validate-the-model
    model.eval()
    
    with torch.no_grad():
        for data, target in valid_loader:
            data = data.to(device)
            target = target.to(device)
            output = model(data)
            output_s = m(output)
            loss = criterion(output_s, target)
        
            predicted = output_s > 0.5
            total_test += target.size(0)
            correct_test += (predicted == target).sum().item()
            y_prob_all_test.append(output_s.cpu().numpy())
            y_true_all_test.append(target.cpu().numpy())
            y_pred_all_test.append(predicted.cpu().numpy())
            # update-average-validation-loss
            valid_loss += loss.item() * data.size(0)
    
    y_prob_test = np.concatenate(y_prob_all_test)
    y_true_test = np.concatenate(y_true_all_test)
    y_pred_test = np.concatenate(y_pred_all_test)
    acc_test = accuracy_score(y_true_test, y_pred_test)
    #perf_test = get_accuracy_(y_true_all_test, y_prob_all_test)
    perf_test = 100 * correct_test / total_test
    print('Test accuracy', acc_test)
    
    # calculate-average-losses
    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(valid_loader.sampler)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)

    # print-training/validation-statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \t Train_perf: {:.6f}\t Test_perf: {:.6f}'.format(
        epoch, train_loss, valid_loss, perf_train, perf_test))

In [ ]:
torch.save(model.state_dict(), './saved_models/cnn_tf_model_442_cnv2D.pth')

In [ ]:
# test-the-model
from sklearn.metrics import f1_score, confusion_matrix

model.eval()  # it-disables-dropout

with torch.no_grad():
    correct = 0
    total = 0
    y_pred_all_eval, y_true_all_eval, y_score_all_eval = list(), list(), list()
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        print('Labels', labels, labels.shape)
        outputs = model(images)
        output_s = m(outputs)
        #print('Output', outputs, outputs.shape)
        #_, predicted = torch.max(outputs.data, 1)
        predicted = output_s > 0.5
        print('PRedicted', predicted, predicted.shape)
        print('check shape', images.shape, labels.shape, outputs.shape, output_s.shape)
        #print('Predicted', predicted, predicted.shape)
        #total += labels.size(0)
        #correct += (predicted == labels).sum().item() 
        y_pred_all_eval.append(predicted.cpu().numpy())
        y_true_all_eval.append(labels.cpu().numpy())
        
    y_pred = np.concatenate(y_pred_all_eval)
    y_true = np.concatenate(y_true_all_eval)
    #y_score = np.concatenate(y_score_all)
    
    #f1_score_eval = f1_score(torch.squeeze(labels).numpy(), torch.squeeze(predicted).numpy())
    f1_score_eval = f1_score(y_true,y_pred)
    confusion_matrice = confusion_matrix(y_true,y_pred)
    accuracy_eval = accuracy_score(y_true,y_pred)
    print('f1_score', f1_score_eval, 'Accuracy', accuracy_eval, 'Confusion matrix', confusion_matrice)
    #print('Test Accuracy of the model: {} %'.format(100 * correct / total))
# Save 
#torch.save(net.state_dict(), './saved_models/cnn_tf_model_442.pth')

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.plot(train_losses, label='Training loss')
plt.plot(valid_losses, label='Validation loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(frameon=False)

In [ ]:
def load_ckp(checkpoint_fpath):
    
    # load check point on CPU
    device = torch.device('cpu')
    model = IEEGSeizureDetection_cnv()
    model.load_state_dict(torch.load(checkpoint_fpath, map_location=device))
    print(device)
    return model

In [ ]:
ckp_path ='./saved_models/cnn_tf_model_442_cnv2D.pth'
#print(model)
fc = load_ckp(ckp_path)
fc = fc
fc.eval()
print(next(fc.parameters()).device)
print(fc)

In [ ]:
#Export to ONNX : GPU-trained model to be loaded using the CPU.
from brevitas.export import FINNManager
from brevitas.export.onnx.generic.manager import BrevitasONNXManager
import netron

FINNManager.export(fc, input_shape=(1, 1, 28, 28), export_path='./saved_models/ONNX_CNN_TF_442_cnv2D.onnx')
#netron.start('./saved_models/ONNX_CNN_TF_442.onnx')

In [ ]:
###### RUN
#optimizer = torch.optim.Adam(net.parameters(), lr=1e-2, betas=(0.9, 0.999))
optimizer = torch.optim.Adam(net.parameters(), lr= 3.0e-3, betas=(0.9, 0.999))
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 
                                                        T_max=4690, 
                                                        eta_min=0, 
                                                        last_epoch=-1)

num_epochs = 50
loss_hist = []
acc_hist = []
correct = 0
total = 0
# training loop
for epoch in range(num_epochs):
    for i, (data, targets) in enumerate(iter(train_loader)):
        data = data.to(device)
        #print('Before',data)
        targets = targets.to(device)
        data = spikegen.rate(data, num_steps=num_steps)
        #print('After:', data)

        net.train()
        spk_rec = forward_pass(net, data)
        loss_val = loss_fn(spk_rec, torch.squeeze(targets,1).type(torch.LongTensor))

        # Gradient calculation + weight update
        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()
        scheduler.step()
        
        # Store loss history for future plotting
        loss_hist.append(loss_val.item())
 
        print(f"Epoch {epoch}, Iteration {i} \nTrain Loss: {loss_val.item():.2f}")
        _, idx = spk_rec.sum(dim=0).max(1)
        print('target', torch.squeeze(targets,1), 'prediction', idx)
        target_for_comp =  torch.squeeze(targets,1)
        accuracy = np.mean((target_for_comp == idx).detach().cpu().numpy())
        temp = (target_for_comp == idx).detach().cpu().numpy()
        print('Mean accuracy from sum', np.mean((target_for_comp == idx).detach().cpu().numpy()))
         #64*64
        #acc = SF.accuracy_rate(spk_rec, targets) 
        acc_hist.append(accuracy)
        print(f"Accuracy: {accuracy* 100:.2f}%\n")
        total += target_for_comp.size(0)
        correct += accuracy * target_for_comp.size(0)
        print('targets.size(0)', target_for_comp.size(0))
        print('Total and correct', total,correct)
        print('Calculated accuracy for an iteration is',100 * correct / total)
        
        # This will end training after 50 iterations by default
        #if i == num_iters:
        #break

In [ ]:
print(net)

In [ ]:
torch.save(net.state_dict(), './saved_models/inference' + '.pth')

In [ ]:
net_inf = nn.Sequential(#qnn.QuantConv2d(1, 16, 5, bias=False, return_quant_tensor=True), #try 16
                    nn.Conv2d(1, 16, 5),
                    nn.BatchNorm2d(16),
                    nn.MaxPool2d(2),
                    snn.Leaky(beta=beta, threshold=1.0, spike_grad=None, init_hidden=True),
                    #qnn.QuantConv2d(16, 64, 5, bias=False,  return_quant_tensor=True),
                    nn.Conv2d(16, 64, 5),
                    nn.BatchNorm2d(64),
                    nn.MaxPool2d(2),
                    snn.Leaky(beta=beta, threshold=1.0, spike_grad=None, init_hidden=True),
                    nn.Flatten(),
                    #qnn.QuantLinear(64 * 4 * 4, 128, bias=False, return_quant_tensor=True),
                    nn.Linear(64*4*4, 128),
                    snn.Leaky(beta=beta, threshold=1.0, spike_grad=None, init_hidden=True),
                    #qnn.QuantLinear(128, 2, bias=False),
                    nn.Linear(128, 2),
                    snn.Leaky(beta=beta, threshold=1.0, spike_grad=None, init_hidden=True, output=True)
                    ).to(device)

In [ ]:
def load_ckp(checkpoint_fpath):
    
    # load check point on CPU
    device = torch.device('cpu')
    model = net_inf
    model.load_state_dict(torch.load(checkpoint_fpath, map_location=device))
    print(device)
    return model

In [ ]:
ckp_path ='./saved_models/inference' + '.pth'
#print(model)
fc = load_ckp(ckp_path)
fc = fc
fc.eval()
print(next(fc.parameters()).device)

In [ ]:
#Export to ONNX : GPU-trained model to be loaded using the CPU.
from brevitas.export import FINNManager
from brevitas.export.onnx.generic.manager import BrevitasONNXManager

FINNManager.export(fc, input_shape=(1, 1, 28, 28), export_path='./saved_models/ONNX_inference.onnx')

In [ ]:
import netron
netron.start('./saved_models/ONNX_inference.onnx')

In [ ]:
#Export to ONNX
FINNManager.export(net, input_shape=(1, 1, 28, 28), export_path='./saved_models/ONNX_' + pat_id + '.onnx')

In [ ]:
from finn.custom_op.base import CustomOp
dir(CustomOp)

In [ ]:
#Print model parameters
#for name, param in net.named_parameters():
#    if param.requires_grad:
#        print(name, param.data)

# Calculate model size
param_size = 0
for param in net.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in net.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))

In [ ]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in net.state_dict():
    print(param_tensor, "\t", net.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

In [ ]:
from sklearn.metrics import f1_score, confusion_matrix

def batch_accuracy(train_loader, net, num_steps):
    with torch.no_grad():
        total = 0
        acc = 0
        net.eval()
        correct = 0
        total = 0
        train_loader = iter(train_loader)
        for data, targets in train_loader:
            data = data.to(device)
            data = spikegen.rate(data, num_steps=num_steps)
            targets = targets.to(device)
            target_for_comp =  torch.squeeze(targets,1)
            #spk_rec, _ = forward_pass(net, num_steps, data)
            spk_rec = forward_pass(net, data)
            
            _, idx = spk_rec.sum(dim=0).max(1)
            
            #acc += SF.accuracy_rate(spk_rec, targets) * spk_rec.size(1)
            #total += spk_rec.size(1)
            accuracy = np.mean((target_for_comp == idx).detach().cpu().numpy())
            #print(f"Accuracy: {accuracy* 100:.2f}%\n")
            total += target_for_comp.size(0)
            correct += accuracy * target_for_comp.size(0)
            print('targets.size(0)', target_for_comp.size(0))
            print('Total and correct', total,correct)
            print('Calculated ACCURACY for an iteration is',100 * correct / total)
            f1_eval = f1_score(target_for_comp, idx, average='weighted')
            print('y_true',target_for_comp,'y_pred',idx)
            #print('Test: Accuracy', acc/total)
            print('F1_SCORE:',f1_eval)
    return correct / total

In [ ]:
test_acc = batch_accuracy(test_loader, net, num_steps)

print(f"The total accuracy on the test set is: {test_acc * 100:.2f}%")

In [ ]:
#RUN
# Plot Loss
acc_hist_epoch = []
acc_hist_epoch = acc_hist[::50]
fig = plt.figure(facecolor="w")
plt.plot(acc_hist_epoch)
plt.title("Train Set Accuracy")
plt.xlabel("iteration")
plt.ylabel("Accuracy")
plt.show()
#For more channels, patients, data!

In [ ]:
'''Experiments:
    1. Test the data trained on another patient (Train on 139 and test on all of 253)
    2. Channel information
    3. Quantisation - check
    4. Visualize Graph ,summary, Layerwise
    '''

In [ ]:
###### RUN
import glob
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader,ConcatDataset
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor

class CustomDataset_reload(Dataset):
    def __init__(self, transform=None):
        #self.imgs_path = "./TF_images_ictal_nonictal/"
        #self.imgs_path = "./TF_plot_images/pat_139/"
        self.imgs_path = "./TF_plot_images_rand/" + pat_id + "/test_data/"
        file_list = glob.glob(self.imgs_path + "*")
        print(file_list)
        self.data = []
        for class_path in file_list:
            class_name = class_path.split("/")[-1]
            for img_path in glob.glob(class_path + "/*.png"):
                self.data.append([img_path, class_name])
        print('self.data length',len(self.data))
        self.class_map = {"non_ictal" : 0, "ictal": 1}
        self.img_dim = (28, 28)
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_path, class_name = self.data[idx]
        #print('img_path', img_path.shape, img_path.type)
        #make it into a function
        '''
        header = ['File Name', 'Pat_ID', 'channel', 'Seizure number','Epoch','Label','Prediction']
        file = open("Eval_plot_data.csv", "w")
        writer = csv.writer(file)
        writer.writerow(header)
        #./TF_plot_images/pat_442/test_data/ictal/EEG_inv_442_HRA4_sNo_17_226_1.png
        pat_id = str(img_path)[-25:-23]
        channel = str(img_path)[-21:-18]
        seizure_n = str(img_path)[-12:-11]
        sample = str(img_path)[-9:-7]
        label_ = str(img_path)[-5]
        for w in range(0,len(self.data)):
            writer.writerow([str(img_path), pat_id,channel, seizure_n, sample, label_ ])
        file.close()
        print('Write to csv', img_path, class_name)
        '''
        
        img = cv2.imread(img_path)                      # Load Image 
        #print('shape for original image',img.shape)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)    # in grayscale
        #img = img[:,:,2]                                # take red component only
        #print('shape for image grayscale',img.shape)
        img = img[60:60+366, 81:81+396]                 # Crop image  
        #print('shape for image cropped',img.shape)
        img = cv2.bitwise_not(img)                      # Negativ-positiv inversion #TODO: improve this
        #print('shape for image bitwise not',img.shape)
        img = cv2.resize(img, self.img_dim)             # resize
        #print('shape for image resize',img.shape)
        #img_tensor = torch.from_numpy(img)
        #img_tensor = torch.unsqueeze(img_tensor, 0).float()
        #print('shape for image before',img_tensor.shape)
        if self.transform is not None:
            img_tensor = self.transform(img)
            img_tensor = img_tensor.float()
            #print('return type of transform',img_tensor.shape, img_tensor.dtype)
        class_id = self.class_map[class_name]
        #if class_id == 1:
        #    cv2.imshow('rgb image', img)
        #    cv2.waitKey(5)
        #print('Label is ', class_id)
        #img_tensor = img_tensor.permute(2, 0, 1)     # for BGR image
        class_id = torch.tensor([class_id])
        #print('data', img_tensor.float().shape , 'targets', class_id.float().shape)
        return img_tensor.float(), class_id.float()

In [ ]:
def plot_confusion_matrix(conf_mat):
    classes_mapping = {0: 'non-ictal', 1: 'ictal'} 
    ticks = list(classes_mapping.keys())
    tick_labels = classes_mapping.values()

    fig, ax = plt.subplots(figsize=(6, 6))
    im = ax.imshow(conf_mat, cmap='Reds')
    ax.set_yticks(ticks)
    ax.set_yticklabels(tick_labels)
    ax.set_xticks(ticks)
    ax.set_xticklabels(tick_labels)
    ax.set_ylabel('True label')
    ax.set_xlabel('Predicted label')
    ax.set_title('Confusion matrix')

    for i in range(len(ticks)):
        for j in range(len(ticks)):
            text = ax.text(
                j, i, conf_mat[i, j], ha='center', va='center', color='k')

    fig.colorbar(im, ax=ax, fraction=0.05, label='# examples')
    fig.tight_layout()
    #fig.savefig('./save_plot/confusion_matrix_'+i+'.png')
    return fig, ax

In [ ]:
import sys
from sklearn.metrics import f1_score, confusion_matrix
np.set_printoptions(threshold=40)

def batch_accuracy_eval(train_loader, net, num_steps):
    with torch.no_grad():
        total = 0
        acc = 0
        net.eval()
        correct = 0
        total = 0
        y_pred_all, y_true_all, f1_all = list(), list(), list()
        train_loader = iter(train_loader)
        for data, targets in train_loader:
            data = data.to(device)
            data = spikegen.rate(data, num_steps=num_steps)
            targets = targets.to(device)
            target_for_comp =  torch.squeeze(targets,1)
            #spk_rec, _ = forward_pass(net, num_steps, data)
            spk_rec = forward_pass(net, data)
            
            _, idx = spk_rec.sum(dim=0).max(1)
            
            #acc += SF.accuracy_rate(spk_rec, targets) * spk_rec.size(1)
            #total += spk_rec.size(1)
            accuracy = np.mean((target_for_comp == idx).detach().cpu().numpy())
            #print(f"Accuracy: {accuracy* 100:.2f}%\n")
            total += target_for_comp.size(0)
            correct += accuracy * target_for_comp.size(0)
            print('targets.size(0)', target_for_comp.size(0))
            print('Total and correct', total,correct)
            print('Calculated ACCURACY for an iteration is',100 * correct / total)
            f1_eval = f1_score(target_for_comp, idx, average='binary')
            print('y_true',target_for_comp,'y_pred',idx)
            
            y_pred_all.append(idx)
            y_true_all.append(target_for_comp)
            #f1_all.append(f1_eval)
            #print('Test: Accuracy', acc/total)
            print('F1_SCORE:',f1_eval)
    
    y_pred = np.concatenate(y_pred_all)
    y_true = np.concatenate(y_true_all)
    print('y_true', y_true, 'y_pred', y_pred)
    f1_comp = f1_score(y_true, y_pred)
    print('Check dimensions', y_pred.shape, y_true.shape)
    print('Total f1 score',f1_comp)
    
    conf_mat = confusion_matrix(y_true, y_pred)
    tn, false_positives, fn, tp = conf_mat.ravel()
    print('conf_mat', conf_mat)
    
    plot_confusion_matrix(conf_mat)
    
    return correct / total

In [ ]:
# Load pat_253 dataset
batch_size = 64
device = 'cpu'
num_workers = 0 
num_steps = 50

transform = transforms.Compose([
            #transforms.Resize((32, 32)),
            #transforms.Grayscale(),     #Returns PIL image
            transforms.ToTensor(),
            transforms.Normalize((0,), (1,))  # ToTensor and Normalize work together
        ])

dataset_test = CustomDataset_reload(transform = transform)
test_loader = DataLoader(dataset_test, batch_size= dataset_test.__len__(), shuffle=False, num_workers=num_workers)
print('Total samples', len(test_loader))


In [ ]:
#Save model for patient 139
torch.save(net.state_dict(), './saved_models/TF_ ' + pat_id + '.pth')

In [ ]:
for i, (x, y) in enumerate(test_loader_253):
    print("batch index {}, 0/1: {}/{}".format(
        i, (y == 0).sum(), (y == 1).sum()))

In [ ]:
# Test on entire dataset of pat_253
beta = 0.8
num_steps=50
spike_grad = surrogate.fast_sigmoid(slope=25)
num_bits = 4
'''
net_253 = nn.Sequential(nn.Conv2d(1, 12, 5),
                    nn.BatchNorm2d(12),
                    nn.MaxPool2d(2),
                    snn.Leaky(beta=beta, threshold=1.0, spike_grad=spike_grad, init_hidden=True),
                    nn.Conv2d(12, 64, 5),
                    nn.BatchNorm2d(64),
                    nn.MaxPool2d(2),
                    snn.Leaky(beta=beta, threshold=1.0, spike_grad=spike_grad, init_hidden=True),
                    nn.Flatten(),
                    nn.Linear(64*4*4, 128),
                    snn.Leaky(beta=beta, threshold=1.0, spike_grad=spike_grad, init_hidden=True),
                    nn.Linear(128, 2),
                    snn.Leaky(beta=beta, threshold=1.0, spike_grad=spike_grad, init_hidden=True, output=True)
                    ).to(device)'''

net_253 = nn.Sequential(#qnn.QuantConv2d(1, 16, 5, bias=False, weight_bit_width= num_bits,return_quant_tensor=True), #try 16
                    nn.Conv2d(1, 16, 5),
                    nn.BatchNorm2d(16),
                    nn.MaxPool2d(2),
                    snn.Leaky(beta=beta, threshold=1.0, spike_grad=spike_grad, init_hidden=True),
                    #qnn.QuantConv2d(16, 64, 5, bias=False, weight_bit_width=num_bits, return_quant_tensor=True),
                    nn.Conv2d(16, 64, 5),
                    nn.BatchNorm2d(64),
                    nn.MaxPool2d(2),
                    snn.Leaky(beta=beta, threshold=1.0, spike_grad=spike_grad, init_hidden=True),
                    nn.Flatten(),
                    #qnn.QuantLinear(64 * 4 * 4, 128, bias=False, weight_bit_width=num_bits, return_quant_tensor=True),
                    nn.Linear(64*4*4, 128),
                    snn.Leaky(beta=beta, threshold=1.0, spike_grad=spike_grad, init_hidden=True),
                    #qnn.QuantLinear(128, 2, bias=False, weight_bit_width=num_bits),
                    nn.Linear(128, 2),
                    snn.Leaky(beta=beta, threshold=1.0, spike_grad=spike_grad, init_hidden=True, output=True)
                    ).to(device)

net_253.apply(reset_weights)

net_253.load_state_dict(torch.load('./saved_models/TF_ ' + pat_id + '.pth'))
#model.eval()
test_acc_253 = batch_accuracy_eval(test_loader_253, net_253, num_steps)

print(f"The total accuracy on the test set is: {test_acc_253 * 100:.2f}%")

In [ ]:
#Print model parameters
net_253.load_state_dict(torch.load('./saved_models/TF_ ' + pat_id + '_16.pth'))
for name, param in net_253.named_parameters():
    if param.requires_grad:
        print(name, param.data, param.data.dtype)

In [ ]:
# Calculate model size , check data type of weights
model = net_253
param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))